In [54]:
import pandas as pd

import os
import inspect

import random
import shutil

In [75]:
this_filename = inspect.getframeinfo(inspect.currentframe()).filename
path = os.path.dirname(os.path.abspath(this_filename))

images_path = path+"/../input/covid-chest-xray/images/all_images"
data = pd.read_csv(path+"/../input/covid-chest-xray/metadata.csv")
print(images_path)

/home/petejb88/mlenv/transfer-learning-image-classifier/working/../input/covid-chest-xray/images/all_images


In [119]:
categories = data.finding.unique()
print(categories)

['COVID-19' 'ARDS' 'SARS' 'Pneumocystis' 'Streptococcus' 'No Finding'
 'Chlamydophila' 'E.Coli' 'COVID-19, ARDS' 'Klebsiella' 'Legionella']


In [40]:
print(len(data))
data_both = data[data['finding'] == 'COVID-19, ARDS']
print(len(data_both.index))

372
12


There are 372 pictures total, and 12 with both COVID-19 and ARDS. 
For ease of classification, we remove these 12.

In [41]:
data_unique = data[data['finding'] != 'COVID-19, ARDS']
infections = data_unique.finding.unique()
print(infections)

['COVID-19' 'ARDS' 'SARS' 'Pneumocystis' 'Streptococcus' 'No Finding'
 'Chlamydophila' 'E.Coli' 'Klebsiella' 'Legionella']


ONLY RUN THIS ONCE: create train, valid, and category FOLDERS

In [121]:
new_folders = ['train', 'valid']

def delete_cat_folders(images_path,new_folders):
    for folder in new_folders:
        try:
            shutil.rmtree(images_path+"/../"+folder)
        except:
            print("Folder {} doesn't exist!".format(folder))
    
def make_cat_folders(images_path,categories):
    for folder in new_folders:
        try:
            os.mkdir(images_path+"/../"+folder)
        except:
            print('Folder {} already exists!'.format(folder))
        for i, cat in enumerate(categories):
            try:
                os.mkdir(images_path+"/../"+folder+"/"+str(i))
            except:
                print("Folder {}/{} already exists!".format(folder,cat))
                
def make_cat_dicts(images_path,categories):
    cat_to_names = {}
    names_to_cat = {}
    for i,cat in enumerate(categories):
            cat_to_names[str(i)] = cat
            names_to_cat[cat] = str(i)
            
    return cat_to_names, names_to_cat
            



In [123]:
cat_to_names, names_to_cat = make_cat_dicts(images_path,categories)

['COVID-19' 'ARDS' 'SARS' 'Pneumocystis' 'Streptococcus' 'No Finding'
 'Chlamydophila' 'E.Coli' 'COVID-19, ARDS' 'Klebsiella' 'Legionella']


Now, we shuffle the images into these folders, while classifying them as we go

First, we check to see if any images are missing: (qualitative)

In [96]:
def check_missing(data,images_path):
    not_in_data = []
    in_data = []
    for file in os.listdir(images_path):
        if os.path.isfile(images_path+"/"+file):
            file_data = data[data['filename'] == file]
            if len(file_data) == 0:
                not_in_data.append(file)
            else:
                in_data.append(file)

    return not_in_data, in_data

images = [image for image in os.listdir(images_path) if os.path.isfile(images_path+"/"+image)]
images_not_in_data, in_data = check_missing(data,images_path)
images_in_data = [image for image in images if (image not in not_in_data)]
print("How many in folder: {}, How many not in data: {}, How many in data: {}".format(
    len(images),
    len(images_not_in_data),
    len(images_in_data)))


How many in folder: 357, How many not in data: 6, How many in data: 351


In [126]:
def shuffle_images(data,images_path,p):
    '''
    Shuffle the images into validation and train folders.
    
    Inputs: 
        - p = percentage split into validation
        - data = metadata
        - images = list of images
        - images_path = location of images
    '''
    
    categories = data.finding.unique()
    delete_cat_folders(images_path,['train', 'valid'])
    make_cat_folders(images_path,categories)
    
    images = [image for image in os.listdir(images_path) if os.path.isfile(images_path+"/"+image)]
    valid_nums = random.sample(range(len(images)),int(len(images)*p))
    
    for image in images:
        image_data = data[data['filename']==image]
        if len(image_data) == 0:
            continue
        else:
            image_index = image_data.index[0]
            image_finding = image_data['finding'][image_index]
            image_cat = names_to_cat[image_finding]
            if image_index in valid_nums:
                os.link(images_path+"/"+image, images_path+"/../valid/"+image_cat+"/"+image)
            else:
                os.link(images_path+"/"+image, images_path+"/../train/"+image_cat+"/"+image)
                
shuffle_images(data,images_path,0.2)

In [9]:
image_data = data.loc[data['X_ray_image_name']=='person40_virus_87.jpeg']
image_index = image_data.index
image_label = image_data['Label'][image_index]
image_type = image_data['Dataset_type'][image_index]
print("Image Label: {}".format(image_label))
print("Image Type: {}".format(image_type))


NameError: name 'data' is not defined